In [1]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import math
import librosa

import urllib.request
import json

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd, sklearn

import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile
import wave
import librosa.display
import soundfile as sf
import torch
import sys
import pickle
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import math
import os
import matplotlib.pyplot as plt
import matplotlib.style as ms
from tqdm import tqdm
import pickle
import IPython.display
import random
ms.use('seaborn-muted')
%matplotlib inline

## Audio

In [2]:
audio_data = "Audio/"
audio_preproccess = 'data/Audio_pre/'
annotations_data_by_emotions = "Annotations_by_emotions/"

df_angry = pd.read_csv(annotations_data_by_emotions + 'data_Angry.csv', index_col='ID')
df_disgusted = pd.read_csv(annotations_data_by_emotions + 'data_Disgusted.csv', index_col='ID')
df_happy = pd.read_csv(annotations_data_by_emotions + 'data_Happy.csv', index_col='ID')
df_neutral = pd.read_csv(annotations_data_by_emotions + 'data_Neutral.csv', index_col='ID')
df_sad = pd.read_csv(annotations_data_by_emotions + 'data_Sad.csv', index_col='ID')
df_scared = pd.read_csv(annotations_data_by_emotions + 'data_Scared.csv', index_col='ID')
df_surprised = pd.read_csv(annotations_data_by_emotions + 'data_Surprised.csv', index_col='ID')

In [3]:
df_angry['Emotion'] = 'ang'
df_disgusted['Emotion'] = 'dis' 
df_happy['Emotion'] = 'hap' 
df_neutral['Emotion'] = 'neu' 
df_sad['Emotion'] = 'sad'
df_scared['Emotion'] = 'sca' 
df_surprised['Emotion'] = 'sur'

In [4]:
df_complete = pd.concat([df_happy, df_angry, df_disgusted, df_neutral, df_sad, df_scared, df_surprised])
df_complete.File += '_mic.wav'

In [5]:
df_complete.head()

,File,Start,End,Emotion
ID,,,,
1,10dec_D11_1_mic.wav,104.750,116.500,hap
1,10dec_D11_1_mic.wav,199.300,210.220,hap
1,10dec_D42_1_mic.wav,27.153,29.440,hap
1,10dec_D52_2_mic.wav,27.470,30.460,hap
1,10dec_D61_1_mic.wav,24.641,26.321,hap


In [6]:
df_complete.to_csv('df_complete.csv', index=False)

In [7]:
labels_df = pd.read_csv('df_complete.csv')

In [8]:
sr = 44100
audio_vectors = {}
metadata_vectors = {}
for sess in [1]:  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = 'Audio/'
    orig_wav_files = os.listdir(wav_file_path)
    for orig_wav_file in tqdm(orig_wav_files):
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
            orig_wav_file, file_format = orig_wav_file.split('.')
            for index, row in labels_df[labels_df['File'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name, emotion = row['Start'], row['End'], row['File'], row['Emotion']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
                audio_vectors["{}_".format(index) + truncated_wav_file_name] = (truncated_wav_vector, emotion)
#                 metadata_vectors["{}_".format(index) + truncated_wav_file_name] = emotion
        except KeyboardInterrupt:
            break
        except:
            print('An exception occured for {}'.format(orig_wav_file))
#     with open('audio_vectors/audio_vectors_{}.pkl'.format(sess), 'wb') as f:
#         pickle.dump(audio_vectors, f)

 11%|█▏        | 66/581 [00:01<00:10, 47.10it/s]/home/olegs/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
 13%|█▎        | 77/581 [00:01<00:14, 35.53it/s]

An exception occured for mic_2_failed.xlsx


100%|██████████| 581/581 [00:14<00:00, 41.42it/s]


In [10]:
# audio_vectors

In [11]:
# class AudioDataset